# calculate other_changes

In [ ]:
import pandas as pd
import numpy as np

# Load your data with MultiIndex columns
df = pd.read_excel("output_cash_flow/all_update_combined.xlsx", header=[0,1])

# Separate the two blocks
df_values = df["values"].copy()
df_reasoning = df["reasoning"].copy()

# Clean column names in values block
df_values.columns = df_values.columns.str.strip()

# Convert numeric columns properly
df_values = df_values.apply(
    lambda col: pd.to_numeric(col, errors="coerce") if col.dtype != "object" else col
)

# -------------------------
# Computations
# -------------------------

# These 3 fields were totaled to get 'long_term_debt_principal_payments' if it was empty
# But these 3 fields are no longer required because extraction result for 'long_term_debt_principal_payments' is good enough
# def compute_long_term_debt_principal_payments(row):
#     original = row.get("long_term_debt_principal_payments")
    
    # bonds = row.get("payments_on_bonds_payable", 0)
    # notes = row.get("payments_on_notes_payable", 0)
    # lease = row.get("payments_on_lease_liabilities", 0)

    # bonds = 0 if pd.isna(bonds) else bonds
    # notes = 0 if pd.isna(notes) else notes
    # lease = 0 if pd.isna(lease) else lease

    # total = bonds + notes + lease

    # if total != 0:
    #     return total, True
    # else:
    #     return original, False

# df_values[["long_term_debt_principal_payments", "computed_flag"]] = df_values.apply(
#     compute_long_term_debt_principal_payments, axis=1, result_type="expand"
# )

# A. Compute other_changes_in_investment_activities
def compute_other_changes(row):
    net_cash = row["net_cash_from_investment_activities"]
    capex = row["capital_expenses"]
    return (0 if pd.isna(net_cash) else net_cash) + (0 if pd.isna(capex) else capex)

df_values["other_changes_in_investment_activities"] = df_values[
    "other_changes_in_investment_activities_calculated"
]

# B. Compute other_changes_in_operating_activities
def compute_operating_changes(row):
    net_cash = row["net_cash_from_operating_activities"]
    net_assets = row["total_change_in_net_assets"]
    non_cash = row["total_non_cash_exp"]
    working_cap = row["change_in_working_capital"]

    return (0 if pd.isna(net_cash) else net_cash) - (
        (0 if pd.isna(net_assets) else net_assets) +
        (0 if pd.isna(non_cash) else non_cash) +
        (0 if pd.isna(working_cap) else working_cap)
    )

df_values["other_changes_in_operating_activities"] = df_values.apply(
    compute_operating_changes, axis=1
)


# C. Compute other_changes_in_financing_activities
# C.1. Compute long_term_debt_principal_payments
#       long_term_debt_principal_payments has not yet included payments_on_lease_liabilities
#       They are extracted separately and were shown to not overlap
df["long_term_debt_principal_payments"] = (
    (df["long_term_debt_principal_payments"].fillna(0))
    + (df["payments_on_lease_liabilities"].fillna(0))
)

# C.2 Compute change_in_long_term_debt
def compute_change_in_long_term_debt(row):
    proceeds = row["long_term_debt_net_proceeds"]
    payments = row["long_term_debt_principal_payments"]
    return (0 if pd.isna(proceeds) else proceeds) + (0 if pd.isna(payments) else payments)

df_values["change_in_long_term_debt"] = df_values.apply(compute_change_in_long_term_debt, axis=1)

# C.3 Compute other_changes_in_financing_activities
df_values["other_changes_in_financing_activities"] = (
    df_values["net_cash_from_financing_activities"] - df_values["change_in_long_term_debt"]
)



# -------------------------
# Column order
# -------------------------
col_order = [
    "School", 
    "Year", 
    "total_change_in_net_assets",
    "total_non_cash_exp",
    "change_in_working_capital",
    "other_changes_in_operating_activities",
    "net_cash_from_operating_activities",
    "capital_expenses",
    "other_changes_in_investment_activities",
    "net_cash_from_investment_activities",
    "long_term_debt_net_proceeds",
    "payments_on_bonds_payable",
    "payments_on_notes_payable",
    "payments_on_lease_liabilities",
    "long_term_debt_principal_payments",
    "change_in_long_term_debt",
    "other_changes_in_financing_activities",
    # "cash_flows_from_noncapital_financing_activities",
    # "cash_flows_from_capital_and_related_financing_activities",
    "net_cash_from_financing_activities",
    "change_in_cash_and_equivalents",
    "cash_flow_2024_unit_multiplier",
    "other_changes_in_investment_activities_calculated",
    "computed_flag"
]

# Reorder values block safely
df_values = df_values.reindex(columns=[c for c in col_order if c in df_values.columns])

# -------------------------
# Recombine values + reasoning
# -------------------------
df_combined = pd.concat({"values": df_values, "reasoning": df_reasoning}, axis=1)

# -------------------------
# Save results
# -------------------------
df_combined.to_excel("output_cash_flow/all_update_combined_filled.xlsx")

print("Saved with values + reasoning:", "output_cash_flow/all_update_combined_filled.xlsx")

Saved with values + reasoning: output_cash_flow/all_update_combined_filled.xlsx
